In [ ]:
import os
import pandas as pd
import Config
# ----------------------------
# CONFIG
# ----------------------------
PERSON_FOLDERS_DIR = r'{}\Working environment'.format(Config.BASE_PATH)
NUM_PERSONS = 36
SUBFOLDER_BRONZE = "Bronze"
SUBFOLDER_SILVER = "Silver"

# ----------------------------
# Helper function: combine tabs per service
# ----------------------------
def process_person_folder(person_folder):
    bronze_folder = os.path.join(person_folder, SUBFOLDER_BRONZE)
    silver_folder = os.path.join(person_folder, SUBFOLDER_SILVER)
    os.makedirs(silver_folder, exist_ok=True)

    # Dictionary to hold combined service data
    service_dict = {}

    # Process all Excel files in Bronze
    for file in os.listdir(bronze_folder):
        if file.endswith(".xlsx"):
            file_path = os.path.join(bronze_folder, file)
            xls = pd.ExcelFile(file_path)

            # Iterate over sheets (services)
            for sheet_name in xls.sheet_names:
                df = pd.read_excel(xls, sheet_name=sheet_name)
                df.to_excel(os.path.join(silver_folder, f"{sheet_name}{file}.xlsx"))
                # Combine same service across files
                if sheet_name in service_dict:
                    service_dict[sheet_name] = pd.concat([service_dict[sheet_name], df], ignore_index=True)
                else:
                    service_dict[sheet_name] = df

    # Save combined service files into Silver folder
    for service, df_combined in service_dict.items():
        service_file = os.path.join(silver_folder, f"{service}.xlsx")
        df_combined.to_excel(service_file, index=False)

    print(f"Processed {person_folder} → Silver folder done.")

# ----------------------------
# Process all 36 persons
# ----------------------------
for person_id in range(1, NUM_PERSONS + 1):
    person_folder = os.path.join(PERSON_FOLDERS_DIR, f"Person_{person_id}")
    process_person_folder(person_folder)

print("\nAll 36 person folders processed successfully. Silver folders created with combined service files.")
